# NiN

## NiN块

NiN块以⼀个普通卷积层开始，后⾯是两个$1 \times 1$的卷积层。这两个$1 \times 1$卷积层充当带有ReLU激活函数的逐像素全连接层。第⼀层的卷积窗口形状通常由⽤⼾
设置，随后的卷积窗口形状固定为$1 \times 1$。


所以NiN块可以定义为：

In [1]:
import torch
import torch.nn as nn

def nin(in_channels, out_channels, kernel_size, strides, padding):
    nin_block = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU())
    return nin_block

NiN块和NiN的[架构](../image/NiN.png)如下：
<div align="center">
<img src = "https://s2.loli.net/2022/04/12/MrzfdjGgRYlWV7k.png" width="50%">
<br>VGG块及其网络架构（左）NiN块及其网络架构（右）
</div>

基于pytorch实现NiN网络的模型搭建如下：

In [2]:
in_channels = 1
num_classes = 10

nin_net = nn.Sequential(
    nin(in_channels, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Dropout(p=0.5),
    nin(384, num_classes, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten())

In [3]:
X = torch.randn(size=(in_channels, 1, 224, 224))
net = nin_net
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape: \t',X.shape)

Sequential output shape: 	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape: 	 torch.Size([1, 96, 26, 26])
Sequential output shape: 	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape: 	 torch.Size([1, 256, 12, 12])
Sequential output shape: 	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape: 	 torch.Size([1, 384, 5, 5])
Dropout output shape: 	 torch.Size([1, 384, 5, 5])
Sequential output shape: 	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape: 	 torch.Size([1, 10, 1, 1])
Flatten output shape: 	 torch.Size([1, 10])


NiN使用分别使用了$11\times 11$、$5\times 5$和$3\times 3$的卷积核。每个NiN块后有一个最大池化层，核为$3\times 3$，步幅为2。

NiN和AlexNet之间的一个显著区别是NiN完全取消了全连接层。NiN使用的是一个NiN块，其输出通道数等于标签类别的数量。最后放一个*全局平均汇聚层*，生成一个*对数几率*。

NiN设计的一个优点是，它显著减少了模型所需参数的数量，但是有时会增加训练模型的时间。


## 模型训练
*后补*

## 模型测试
*后补*